In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.metrics import accuracy_score, classification_report

from joblib import dump, load

from reportlab.pdfgen import canvas

## Load Model

In [2]:
# Load the model from the file
loaded_model = load('RiskPredictionArtifacts/model.joblib')

## Load Test Patients Data

In [7]:
X_test_5 = pd.read_csv("RiskPredictionArtifacts/X_test_5patients.csv")
y_test_5 = pd.read_csv("RiskPredictionArtifacts/y_test_5patients.csv")

In [8]:
X_test_5_final = X_test_5.drop(columns=['id'])

## Make Predictions and Evaluate

In [9]:
# Make predictions on the test set
y_pred = loaded_model.predict(X_test_5_final)

# Evaluate the model
accuracy = accuracy_score(y_test_5, y_pred)
print(f"Accuracy: {accuracy}")

# View detailed classification report
print(classification_report(y_test_5, y_pred))

Accuracy: 0.8
              precision    recall  f1-score   support

           0       0.75      1.00      0.86         3
           1       1.00      0.50      0.67         2

    accuracy                           0.80         5
   macro avg       0.88      0.75      0.76         5
weighted avg       0.85      0.80      0.78         5



## Export as pdf

In [102]:
y_test_df = pd.DataFrame(y_test_5)
y_pred_df = pd.DataFrame(y_pred,  columns=['pred'])

In [103]:
merged_df = pd.merge(pd.merge(X_test_5, y_test_df, left_index=True, right_index=True), y_pred_df, left_index=True, right_index=True)

In [104]:
merged_df.shape

(5, 188)

In [105]:
merged_df.head()

,id,state,race,ethnicity,Otitis media,Fever (finding),Suspected COVID-19,COVID-19,Sprain of ankle,Cough (finding),...,Unnamed: 178,Unnamed: 179,Unnamed: 180,Unnamed: 181,Unnamed: 182,Unnamed: 183,Unnamed: 184,Unnamed: 185,risk,pred
0,432d95bf-19e9-4c20-96bd-f7a84dd92aa7,1,5,1,0.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
1,6fd20c3b-de36-4ba9-86ad-4a64e9d53781,1,5,1,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
2,3e8f1d67-152e-411b-b9b7-c0c232ea6626,1,5,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,4,0,0
3,5475def7-5a54-4f18-99b8-2d20a371a92c,1,5,1,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,4,1,1
4,5cc75ac6-5bb0-44fd-9f35-5e840766fd1b,1,5,1,0.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,4,1,0


In [106]:
# Create a PDF file
pdf_filename = "output.pdf"
c = canvas.Canvas(pdf_filename)

# Set initial y position
y_position = 750

# Write text to the PDF
for index, row in merged_df.iterrows():
    if (row['pred'] == 0):
        text = "Patient " + row['id'] + " is at low risk of mortality from Covid19"
    else:
        text = "Patient " + row['id'] + " is at high risk of mortality from Covid19"
    
    c.drawString(100, y_position, text)
    y_position -= 20

# Save the PDF file
c.save()
